Coding a Predict model to predict whether the passanger of the titanic will survive or not

In [ ]:
# Installing necessary libraries
!pip install -U scikit-learn

In [49]:
# Importing necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score

In [36]:
# Reading train dataset and taking a first look at it
dataset = pd.read_csv(r'train.csv')

dataset.info() #Columns with null values: age, cabin, embarked
# dataset.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [ ]:
# Taking a look at the first lines of the dataset
# According to Kaggle, the columns survived is the one that we gonna use to train our predict model
dataset.head()

In [51]:
# Filling the NaN values with median of the own column
# Filling Cabin and Embarked NaN values with a new category: Unknown
dataset['Age'] = dataset['Age'].fillna(dataset['Age'].median())
dataset['Age'] = dataset['Age'].fillna('Unknown')
dataset['Age'] = dataset['Age'].fillna('Unknown')